In [2]:
import logging

logging.basicConfig(
    format="%(asctime)s %(levelname)s [%(name)s] %(message)s", level=logging.INFO, datefmt="%Y-%m-%d %H:%M:%S"
)
logging.getLogger("py4j").setLevel(logging.WARNING)
logging.getLogger("sh.command").setLevel(logging.ERROR)

# COMMAND ----------

import os
import re
import training.consts as consts
from datetime import datetime
from training.consts import DEFAULT_INPUT_MODEL, SUGGESTED_INPUT_MODELS
from training.trainer import load_training_dataset, load_tokenizer
from training.generate import generate_response, load_model_tokenizer_for_generate

In [3]:
model, tokenizer = load_model_tokenizer_for_generate("/home/mcwave/data/models/falcon-7b")

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Model path:/home/mcwave/data/models/falcon-7b


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
problems=pd.read_parquet("/home/mcwave/data/word_problems_raw/gsm8k-train.parquet", engine='pyarrow')
def compare(problems,model,tokenizer,n):
    letters="qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM"
    for i in range(n):
        q=problems["question"][i]
        a=problems["answer"][i]
        res=""
        j=len(a)-1
        while a[j]!=" ":
            res=a[j]+res
            j-=1
        #print(res)
        response=generate_response(q,model=model,tokenizer=tokenizer)
        print(response)
        j=0
        found=False
        while j<len(response):
            title=""
            if j<len(response)-1 and response[j]=="#" and response[j+1]==" ":
                j+=2
                while j<len(response) and response[j] in letters:
                    title+=response[j]
                    j+=1
            if title in ["Response","response","Solution","solution","Answer","answer"]:
                ans=""
                while j<len(response):
                    add=1
                    while j<len(response) and response[j] in "0123456789":
                        ans+=response[j]
                        add=0
                        j+=1
                    j+=add
                print(ans)
                if ans==res:
                    print("correct")
                    found=True
                    break
                else:
                    print("incorrect")
                    found=True
                    break
            j+=1
        if not found:
            num=""
            j=0
            printed=False
            while j<len(response):
                add=1
                while j<len(response) and response[j] in "0123456789.":
                    add=0
                    num+=response[j]
                    j+=1
                j+=add
                if add==0 and num!=".":
                    if num[-1]==".":
                        num=num[:-1]
                    if num==res:
                        print("correct")
                        break
                    else:
                        if not printed:
                            print("incorrect")
                            printed=True
            j=len(response)-1
            while j>=0:
                add=1
                while j>=0 and response[j] in "0123456789.":
                    add=0
                    num+=response[j]
                    j-=1
                j-=add
                if add==0 and num!=".":
                    if num[-1]==".":
                        num=num[:-1]
                    if num==res:
                        print("correct")
                        break
                    else:
                        if not printed:
                            print("incorrect")
                            printed=True
compare(problems,model,tokenizer,10)

In [1]:
from transformers import AutoTokenizer
import transformers
import torch

model = "/home/mcwave/models/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

/home/mcwave/anaconda3/envs/torch200/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:06<00:00,  3.11s/it]
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2

In [2]:
sequences = pipeline(
   "Write a poem about Valencia.",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

/home/mcwave/anaconda3/envs/torch200/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


AttributeError: module 'torch.nn.functional' has no attribute 'scaled_dot_product_attention'

In [9]:
problems["answer"][0]

'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'